In [1]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import plotly.express as px

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image


In [2]:
# dataset file path

malignant_files = "dataset/trial/Malignant"
normal_files = "dataset/trial/Normal"

In [3]:
# function to load the images from the directories

def load_images(directory, label):
  images = []
  labels = []
  for filename in os.listdir(directory):
    if filename.endswith(".jpg") or filename.endswith(".png"):
      img_path = os.path.join(directory, filename)
      img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
      if img is not None:
        img = cv2.resize(img, (128, 128))
        images.append(img)
        labels.append(label)
  return images, labels

In [4]:
# normal = 0
# malignant = 1

malignant_images, malignant_labels = load_images(malignant_files, label=1)
normal_images, normal_labels = load_images(normal_files, label=0)

In [5]:
images = malignant_images + normal_images
labels = malignant_labels + normal_labels

In [6]:
images = np.array(images)
labels = np.array(labels)


print(len(images))

971


In [7]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [8]:
categories = {
    "Malignant": malignant_files,
    "Normal": normal_files
}

In [12]:
model = Sequential([
  Input(shape=(128, 128, 1)),
  Conv2D(32, (3, 3), activation = 'relu'),
  MaxPooling2D((2,2)),
  Conv2D(64, (3, 3), activation = 'relu'),
  MaxPooling2D((2,2)),
  Conv2D(128, (3, 3), activation = 'relu'),
  MaxPooling2D((2,2)),
  Flatten(),
  Dense(512, activation = 'relu'),
  Dropout(0.5),
  Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

y_train_encoded = to_categorical(y_train, num_classes = 2)
y_test_encoded = to_categorical(y_test, num_classes = 2)

print("Shape of X_train : ", x_train.shape)
print("Shape of y_train_encoded : ", y_train_encoded)

history = model.fit(x_train, y_train_encoded, epochs=10, batch_size=32, validation_split=0.2)

print("Training history : ", history.history)

Shape of X_train :  (776, 128, 128)
Shape of y_train_encoded :  [[1. 0.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [0. 1.]
 [0. 1.]]
Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.5251 - loss: 2146.7822 - val_accuracy: 0.8205 - val_loss: 60.0619
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.6615 - loss: 247.7907 - val_accuracy: 0.9551 - val_loss: 7.2765
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.8900 - loss: 22.8400 - val_accuracy: 0.9872 - val_loss: 1.7333
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.9433 - loss: 5.9265 - val_accuracy: 0.9744 - val_loss: 3.0338
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.9832 - loss: 2.2727 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 20s 981ms/step - accuracy: 0.9922 - loss: 0.1646 - val_accuracy: 1.0000 - val_loss: 9.5955e-04
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 20s 995ms/step - accuracy: 0.9964 - loss: 0.0510 - val_accuracy:


KeyboardInterrupt



In [10]:
y_test_encoded = to_categorical(y_test, num_classes=2)

test_loss, test_acc = model.evaluate(x_test, y_test_encoded)
#print(f'Test Loss: {test_loss}')
#print(f'Test Accuracy: {test_acc}')

print("Model Accuracy =",test_acc*100)

7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - accuracy: 1.0000 - loss: 0.0055
Model Accuracy = 100.0


In [ ]:
def load_and_preprocess_image(img_path, target_size=(128, 128)):
    #print(img_path)
    img = image.load_img(img_path, target_size=target_size, color_mode='grayscale')
    img_array = image.img_to_array(img)
    plt.imshow(img_array.squeeze(), cmap='gray')
    plt.title('CT Scan')
    plt.axis('off')  # Hide axis
    plt.show()
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0
    return img_array

In [ ]:
def predict_image(model, img_path):
  img_array = load_and_preprocess_image(img_path)
  prediction = model.predict(img_array)
  class_index = np.argmax(prediction, axis = 1)
  return class_index[0]

In [ ]:
class_labels = ['Normal', 'Malignant']

test_img = 'dataset/trial/test/m2.jpg'

In [ ]:
predicted_class_index = predict_image(model, test_img)
predicted_class = class_labels[predicted_class_index]

print(f'The model predicts that the image as : {predicted_class}')

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()

In [ ]:
def load_single_image(img_path):
    # Load the image in grayscale
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        # Resize the image to the required input size
        img = cv2.resize(img, (128, 128))
        # Normalize the image data to the range [0, 1]
        img = img / 255.0
        # Expand dimensions to match the model input shape
        img = np.expand_dims(img, axis=-1)  # Add channel dimension
        img = np.expand_dims(img, axis=0)   # Add batch dimension

        # Display the image using matplotlib
        plt.imshow(img[0, :, :, 0], cmap='gray')
        plt.title("Loaded Image")
        plt.axis('off')
        plt.show()
    
    return img

In [ ]:
# Example usage:
img_path = 'dataset/trial/test/n3.jpg'
image = load_single_image(img_path)

# Now you can predict using your model
prediction = model.predict(image)
predicted_label = np.argmax(prediction)

predicted_class = class_labels[predicted_label]

print("Predicted label:", predicted_class)